In [2]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Cargar embeddings y el índice FAISS
nodos_embeddings = np.load('nodos_embeddings.npy')
index = faiss.read_index('nodos_index.faiss')

In [ ]:
# Inicializar el modelo
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Lista de nodos (asegúrate de que coincide con los datos de los embeddings)
nodos = ["nodos_index.faiss"]

In [9]:

# Lista de nodos (asegúrate de que coincide con los datos de los embeddings)
nodos = ["nodo1", "nodo2", "nodo3", "..."]

In [17]:
# Definir la función de consulta
def consulta_rag(consulta, index, nodos, model, k=5):
    consulta_embedding = model.encode([consulta])
    D, I = index.search(np.array(consulta_embedding), k)
    
    print(f"Consulta: {consulta}")
    for i in range(k):
        print(f"Resultado {i + 1}: {nodos[I[0][i]]} (Distancia: {D[0][i]:.4f})")

# Ejemplo de consulta
consulta_rag("arroz", index, nodos, model)

Consulta: arroz


IndexError: list index out of range

In [18]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Cargar embeddings y el índice FAISS
nodos_embeddings = np.load('nodos_embeddings.npy')
index = faiss.read_index('nodos_index.faiss')

# Inicializar el modelo
model = SentenceTransformer('all-MiniLM-L6-v2')

# Lista de nodos (asegúrate de que coincide con los datos de los embeddings)
nodos = ["nodo1", "nodo2", "nodo3", "..."]  # Asegúrate de que esta lista tenga el mismo tamaño que nodos_embeddings

# Definir la función de consulta
def consulta_rag(consulta, index, nodos, model, k=5):
    k = min(k, len(nodos))  # Limitar k al número de nodos disponibles
    consulta_embedding = model.encode([consulta])

    # Asegurarse de que el embedding tenga la forma correcta (1, embedding_dimension)
    consulta_embedding = np.array(consulta_embedding).reshape(1, -1)
    D, I = index.search(consulta_embedding, k)
    
    print(f"Consulta: {consulta}")
    print("Distancias (D):", D)  # Imprime las distancias obtenidas
    print("Índices (I):", I)  # Imprime los índices obtenidos

    # Comprobar si los índices están dentro del rango de nodos
    for i in range(k):
        if I[0][i] < len(nodos):
            print(f"Resultado {i + 1}: {nodos[I[0][i]]} (Distancia: {D[0][i]:.4f})")
        else:
            print(f"Resultado {i + 1}: Índice fuera de rango")

# Ejemplo de consulta
consulta_rag("arroz", index, nodos, model)


Consulta: arroz
Distancias (D): [[2.1730066e-13 6.2418938e-01 1.0416648e+00 1.0424937e+00]]
Índices (I): [[18136 14780     0 25986]]
Resultado 1: Índice fuera de rango
Resultado 2: Índice fuera de rango
Resultado 3: nodo1 (Distancia: 1.0417)
Resultado 4: Índice fuera de rango


In [20]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from collections import Counter

# Cargar embeddings y el índice FAISS
nodos_embeddings = np.load('nodos_embeddings.npy')
index = faiss.read_index('nodos_index.faiss')

# Inicializar el modelo
model = SentenceTransformer('all-MiniLM-L6-v2')

# Lista de nodos (asegúrate de que coincide con los datos de los embeddings)
nodos = ["Soja", "Sorgo", "Arveja", "Cercospora sojina", "..."]  # Lista completa de nodos

# Definir la función de consulta
def consulta_rag(consulta, index, nodos, model, k=10):
    # Limitar k al número de nodos disponibles
    consulta_embedding = model.encode([consulta])
    consulta_embedding = np.array(consulta_embedding).reshape(1, -1)
    D, I = index.search(consulta_embedding, k)
    
    print(f"Consulta: {consulta}")

    # Crear una lista de resultados con nombres de nodos y distancias
    resultados = [(nodos[I[0][i]], D[0][i]) for i in range(k) if I[0][i] < len(nodos)]

    # Agrupar y contar las ocurrencias de cada nodo
    resultados_agrupados = Counter(resultados)
    
    # Mostrar los resultados agrupados y ordenados por distancia
    for i, ((nombre, distancia), count) in enumerate(resultados_agrupados.most_common(), start=1):
        print(f"Resultado {i}: {nombre} (Distancia: {distancia:.4f})")
        if count > 1:
            for _ in range(count - 1):
                i += 1
                print(f"Resultado {i}: {nombre} (Distancia: {distancia:.4f})")
        if i >= k:
            break

# Ejemplo de consulta
consulta_rag("soja", index, nodos, model)


Consulta: soja
